In [19]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

In [20]:
tf.__version__

'2.10.0'

# Create a Submission File 

The submitted file must include all the predictions from Dan, Kyu, and Play Style testing CSVs. Therefore, you must train your models before submitting any file. If your model is not trained for Dan, Kyu, or Play Style, check out the previous tutorials about how to train your model or just simply use the baseline models (model_dan_tutorial.h5, model_kyu_tutorial.h5, model_playstyle.h5) 

**PublicUpload.csv** must be in the following form:
```
DTPU0000000001,id,qr,pq,pd,ab
DTPU0000000002,ao,ab,ha,ff,qd
DTPU0000000003,qd,gd,fh,ed,fa
DTPU0000000004,pr,ba,dq,hh,aj
DTPU0000000005,ph,jh,af,df,gj
.
.
.
KTPU0000000001,id,qr,pq,pd,ab
KTPU0000000002,ao,ab,ha,ff,qd
KTPU0000000003,qd,gd,fh,ed,fa
KTPU0000000004,pr,ba,dq,hh,aj
KTPU0000000005,ph,jh,af,df,gj
.
.
.
PSTPU0000000001_79,2
PSTPU0000000001_55,3
PSTPU0000000001_105,2
PSTPU0000000002_37,1
PSTPU0000000002_113,3
```

**For Dan and Kyu:**
- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

**For PlayStyle**:
- Column 1: Game ID
- Column 2: Predicted Game Style

The code block below is copied from the tutorial notebooks and it includes the pre-processing methods of the trained models. You might need to change the functions below with respect to your models.

In [21]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}

def prepare_input_for_dan_kyu_models(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_input_for_playstyle_model(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[last_move_row,last_move_column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x
    # x = np.zeros((19,19,2))
    # if moves and len(moves[0]) < 2:
    #     return x
    # for move in moves:
    #     color = move[0]
    #     column = coordinates[move[2]]
    #     row = coordinates[move[3]]
    #     x[row,column,0] = 1
    # if moves:
    #     last_move_column = coordinates[moves[-1][2]]
    #     last_move_row = coordinates[moves[-1][3]]
    #     x[row,column,1] = 1
    # return x

def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

## Save Predictions for Kyu
The same code block applies to Kyu by changing the model and dataset

In [22]:
# Load your own model. Here we use the baseline model
model = load_model('./kyus/kyu9_0.52325_1.94353_70.h5')

# Load the corresponding dataset
df = open('./CSVs/kyu_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# import os
# from tensorflow.keras.models import load_model

# folder_path = "./kyus/" 

# models = [] 

# for filename in os.listdir(folder_path):
#     if filename.endswith(".h5"):  
#         model_path = os.path.join(folder_path, filename)
#         model = load_model(model_path)
#         models.append(model)
        
# predictions_ensemble = []
# for model in models:
#     predictions = model.predict(x_testing)
#     predictions_ensemble.append(predictions)

# ensemble_mean = np.mean(predictions_ensemble, axis=0)

# prediction_chars = top_5_preds_with_chars(ensemble_mean)


# Save results to public_submission_template.csv
with open('./private_submission.csv','a') as f:
    f.truncate(0)
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

355/355 [==============================] - 2s 4ms/step


## Save Predictions for Dan
The code block below is to use **dan model** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [23]:


# Load the corresponding dataset
df = open('./CSVs/dan_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing = np.array(x_testing)

# single model

model = load_model('./ensemble_models/dan5_40_0.48263_2.14965.h5')
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# # ensemble models

# import os
# from tensorflow.keras.models import load_model

# folder_path = "./ensemble_models/" 

# models = [] 

# for filename in os.listdir(folder_path):
#     if filename.endswith(".h5"):  
#         model_path = os.path.join(folder_path, filename)
#         model = load_model(model_path)
#         models.append(model)
        
# predictions_ensemble = []
# for model in models:
#     predictions = model.predict(x_testing)
#     predictions_ensemble.append(predictions)

# ensemble_mean = np.mean(predictions_ensemble, axis=0)

# prediction_chars = top_5_preds_with_chars(ensemble_mean)


# Save results to public_submission_template.csv
with open('./private_submission.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

344/344 [==============================] - 2s 5ms/step


## Save Predictions for PlayStyle
Playstyle requires only one predictions so the code here is a simple version of above

In [24]:
# Load your own model. Here we use the baseline model
model = load_model('./pl/0.786_0.677.h5')

# Load the corresponding dataset
df = open('./CSVs/play_style_test_private.csv').read().splitlines()
games_id = [i.split(',',1)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]
x_testing = []
count = 0
for game in games:
    moves_list = game.split(',')
    i = len(moves_list) - 1
    while i >= 0:
        if len(moves_list[i]) < 2:
            moves_list = moves_list[:i]
            i = i - 1
        else:
            break
    count += 1
    x_testing.append(prepare_input_for_playstyle_model(moves_list))

x_testing = np.array(x_testing)

# predictions = model.predict(x_testing)
# prediction_numbers = np.argmax(predictions, axis=1)

import os
from tensorflow.keras.models import load_model

folder_path = "./pmodels/" 

# number = 10
models = [] 

for filename in os.listdir(folder_path):
    if filename.endswith(".h5"):  
        model_path = os.path.join(folder_path, filename)
        model = load_model(model_path)
        models.append(model)
    # number -= 1
    # if number == 0:
    #     break
        
predictions_ensemble = []

for model in models:
    prediction = model.predict(x_testing)
    predictions_ensemble.append(prediction)

ensemble_mean = np.mean(predictions_ensemble, axis=0)
prediction_numbers = np.argmax(ensemble_mean, axis=1)

z = 0
o = 0
t = 0
for n in prediction_numbers:
    if n == 0:
        z += 1
    if n == 1:
        o += 1
    else:
        t += 1

print(z, o, t)

with open('./private_submission.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

313/313 [==============================] - 2s 6ms/step
3258 3169 6829


Now you can upload your file from the competition website! (https://tbrain.trendmicro.com.tw/Competitions/Details/29)